In [1]:
import os

In [2]:
%pwd

'e:\\Projects\\Text-Summarization\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'e:\\Projects\\Text-Summarization'

In [5]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class DataValidationConfig:
    root_dir: Path
    STATUS_FILE: str
    ALL_REQUIRED_FILES: list

In [6]:
from textSummarizer.constants import *
from textSummarizer.utils.common import read_yaml, create_directories

In [7]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])


    
    def get_data_validation_config(self) -> DataValidationConfig:
        config = self.config.data_validation

        create_directories([config.root_dir])

        data_validation_config = DataValidationConfig(
            root_dir=config.root_dir,
            STATUS_FILE=config.STATUS_FILE,
            ALL_REQUIRED_FILES=config.ALL_REQUIRED_FILES,
        )

        return data_validation_config


In [8]:
import os
from textSummarizer.logging import logger

In [ ]:
class DataValidation:
    def __init__(self, config: DataValidationConfig):
        self.config = config

    def validate_all_files_exist(self) -> bool:
        try:
            # Path where data was extracted
            data_path = os.path.join("artifacts", "data_ingestion", "samsum_dataset")
            
            # List all files/folders in that directory
            all_files = os.listdir(data_path)
            logger.info(f"Files found in dataset: {all_files}")

            # Find missing files
            missing_files = [file for file in self.config.ALL_REQUIRED_FILES if file not in all_files]

            # Determine validation status
            validation_status = len(missing_files) == 0

            # Write status to file
            with open(self.config.STATUS_FILE, 'w') as f:
                if validation_status:
                    f.write(f"Validation Status: SUCCESS \nAll required files are present.")
                    logger.info("All required files are present.")
                else:
                    f.write(f"Validation Status: FAILED \nMissing files: {missing_files}")
                    logger.error(f"Missing files: {missing_files}")

            return validation_status

        except Exception as e:
            logger.exception(e)
            raise e


In [12]:
try:
    config = ConfigurationManager()
    data_validation_config = config.get_data_validation_config()
    data_validation = DataValidation(config=data_validation_config)
    data_validation.validate_all_files_exist()
except Exception as e:
    raise e


YAML file loaded successfully: config\config.yaml
[2025-11-01 22:56:46,243: INFO: common: YAML file loaded successfully: config\config.yaml]
YAML file loaded successfully: params.yaml
[2025-11-01 22:56:46,249: INFO: common: YAML file loaded successfully: params.yaml]
Created directory at: artifacts
[2025-11-01 22:56:46,253: INFO: common: Created directory at: artifacts]
Created directory at: artifacts/data_validation
[2025-11-01 22:56:46,259: INFO: common: Created directory at: artifacts/data_validation]
Files found in dataset: ['dataset_dict.json', 'test', 'train', 'validation']
[2025-11-01 22:56:46,259: INFO: 3771648450: Files found in dataset: ['dataset_dict.json', 'test', 'train', 'validation']]
All required files are present.
[2025-11-01 22:56:46,267: INFO: 3771648450: All required files are present.]
